In [ ]:
import pandas as pd
import scipy.stats as stats 
import numpy as np

## Network Initialization

In [4]:
adjacency = pd.DataFrame(
    np.array([[1, 1, 1, 1, 0],
              [1, 1, 1, 1, 1],
              [1, 1, 1, 1, 1],
              [1, 1, 1, 1, 1],
              [0, 1, 1, 1, 1]]),
    columns=['1', '2', '3', '4', '5'],
    index=['1', '2', '3', '4', '5']
)
adjacency

,1,2,3,4,5
1,1,1,1,1,0
2,1,1,1,1,1
3,1,1,1,1,1
4,1,1,1,1,1
5,0,1,1,1,1


### Discretize the probability functions of all 9 edges

In [ ]:
def pdf(gamma, alpha, time):
    return ((alpha^gamma)*np.exp(-alpha*time)*time(gamma-1))/((gamma-1)!)

## Simulation